In [8]:
import random

class EpsilonGreedyRecommender:
    def __init__(self, items, epsilon=0.1):
        """
        :param items: lista dostupnih 'krakova' - u tvom slučaju to mogu biti modeli patika
        :param epsilon: verovatnoća za nasumično biranje kraka (exploration)
        """
        self.items = items
        self.epsilon = epsilon
        
        # Broj preporuka (pokušaja) i broj ostvarenih reward-a za svaki item
        self.counts = {item: 0 for item in items}
        self.rewards = {item: 0 for item in items}

    def select_item(self):
        """
        Vrati item (patiku) koji treba da preporučimo korisniku.
        Radi se epsilon-greedy selekcija.
        """
        # Slučaj kada biramo nasumično (exploration)
        if random.random() < self.epsilon:
            return random.choice(self.items)
        
        # Slučaj kada biramo najbolju patiku do sada (exploitation)
        best_item = None
        best_avg_reward = -1
        
        # Proveri (reward / count) za svaki item kako bismo pronašli najbolji
        for item in self.items:
            if self.counts[item] == 0:
                # Ako nikad nismo prikazali taj item, moze se desiti da ga odaberemo
                avg_reward = 0
            else:
                avg_reward = self.rewards[item] / float(self.counts[item])
            
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward
                best_item = item
        
        return best_item

    def update(self, chosen_item, reward):
        """
        Ažuriraj broj pokušaja i ostvarenih reward-a za dati item.
        :param chosen_item: patika koja je bila preporučena
        :param reward: 0 ili 1 u najjednostavnijem slučaju
        """
        self.counts[chosen_item] += 1
        self.rewards[chosen_item] += reward


# ------------------------------
# Primer korišćenja
if __name__ == "__main__":
    # Zamislimo da imamo 3 modela patika
    patike = ["Nike Air Max", "Adidas Ultraboost", "Puma Rider"]

    # Inicijalizuj epsilon-greedy recommender
    recommender = EpsilonGreedyRecommender(items=patike, epsilon=0.1)
    # Simulacija interakcija (npr. 1000 korisnika)

    
    # (Opciono) Zamislimo da svaki model patika ima određenu "pravu" verovatnoću kupovine
    true_conversion_rates = {
        "Nike Air Max": 0.05,       # 5% kupuje
        "Adidas Ultraboost": 0.08,  # 8% kupuje
        "Puma Rider": 0.03          # 3% kupuje
    }

    num_users = 1000
    for _ in range(num_users):
        # 1. Odaberi patiku koju ćemo preporučiti
        chosen = recommender.select_item()
        
        # 2. Simuliraj "reward" – da li je korisnik kupio
        #    U praksi, ovo dolazi iz realnih podataka (klik/buy), ovde samo simuliramo
        prob = true_conversion_rates[chosen]
        reward = 1 if random.random() < prob else 0
        
        # 3. Update-uj bandit na osnovu dobijenog rezultata
        recommender.update(chosen, reward)

    # Nakon 1000 iteracija, možemo pogledati koliko često je prikazan svaki model i kakav je reward
    for item in patike:
        print(f"Patika: {item}")
        print(f"  Ukupno prikazano: {recommender.counts[item]}")
        print(f"  Ukupno kupljeno: {recommender.rewards[item]}")
        if recommender.counts[item] > 0:
            print(f"  Prosečni reward: {recommender.rewards[item] / float(recommender.counts[item]):.3f}")
        print("---")

Patika: Nike Air Max
  Ukupno prikazano: 540
  Ukupno kupljeno: 34
  Prosečni reward: 0.063
---
Patika: Adidas Ultraboost
  Ukupno prikazano: 407
  Ukupno kupljeno: 35
  Prosečni reward: 0.086
---
Patika: Puma Rider
  Ukupno prikazano: 53
  Ukupno kupljeno: 1
  Prosečni reward: 0.019
---


In [9]:
import random
import json
import os

class EpsilonGreedyRecommender:
    def __init__(self, filepath, epsilon=0.1):
        """
        :param filepath: Putanja do JSON fajla sa podacima o patikama
        :param epsilon: Verovatnoća za nasumično biranje kraka (exploration)
        """
        self.filepath = filepath
        self.epsilon = epsilon
        self.items = []
        self.counts = {}
        self.rewards = {}
        self.load_data()
    
    def load_data(self):
        """
        Učitaj podatke o patikama iz JSON fajla.
        Ako fajl ne postoji, inicijalizuj prazan skup patika.
        """
        if not os.path.exists(self.filepath):
            raise FileNotFoundError(f"Fajl {self.filepath} ne postoji.")
        
        with open(self.filepath, 'r') as f:
            data = json.load(f)
            for entry in data:
                name = entry['name']
                self.items.append(name)
                self.counts[name] = entry.get('count', 0)
                self.rewards[name] = entry.get('reward', 0)
    
    def save_data(self):
        """
        Sačuvaj trenutne podatke o patikama u JSON fajl.
        """
        data = []
        for item in self.items:
            data.append({
                "name": item,
                "count": self.counts.get(item, 0),
                "reward": self.rewards.get(item, 0)
            })
        
        with open(self.filepath, 'w') as f:
            json.dump(data, f, indent=4)
    
    def select_item(self):
        """
        Vrati item (patiku) koji treba da preporučiš korisniku.
        Radi se epsilon-greedy selekcija.
        """
        # Slučaj kada biramo nasumično (exploration)
        if random.random() < self.epsilon:
            chosen = random.choice(self.items)
            return chosen
        
        # Slučaj kada biramo najbolju patiku do sada (exploitation)
        best_item = None
        best_avg_reward = -1
        
        for item in self.items:
            if self.counts[item] == 0:
                avg_reward = 0  # Ako nikad nismo prikazali taj item, postavimo avg_reward na 0
            else:
                avg_reward = self.rewards[item] / float(self.counts[item])
            
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward
                best_item = item
        
        return best_item
    
    def update(self, chosen_item, reward):
        """
        Ažuriraj broj pokušaja i ostvarenih reward-a za dati item, i sačuvaj podatke u fajl.
        :param chosen_item: Patika koja je bila preporučena
        :param reward: 0 ili 1 u najjednostavnijem slučaju
        """
        if chosen_item not in self.items:
            raise ValueError(f"Odabrana patika '{chosen_item}' nije poznata.")
        
        self.counts[chosen_item] += 1
        self.rewards[chosen_item] += reward
        self.save_data()


# ------------------------------
# Primer korišćenja
if __name__ == "__main__":
    # Putanja do JSON fajla
    filepath = "sneakers.json"
    
    # Inicijalizuj epsilon-greedy recommender
    recommender = EpsilonGreedyRecommender(filepath=filepath, epsilon=0.1)
    
    # Simulacija interakcija (npr. 1000 korisnika)
    
    # (Opciono) Zamislimo da svaki model patika ima određenu "pravu" verovatnoću kupovine
    true_conversion_rates = {
        "Nike Air Max": 0.05,       # 5% kupuje
        "Adidas Ultraboost": 0.08,  # 8% kupuje
        "Puma Rider": 0.03          # 3% kupuje
    }

    num_users = 1000
    for _ in range(num_users):
        # 1. Odaberi patiku koju ćemo preporučiti
        chosen = recommender.select_item()
        
        # 2. Simuliraj "reward" – da li je korisnik kupio
        #    U praksi, ovo dolazi iz realnih podataka (klik/buy), ovde samo simuliramo
        prob = true_conversion_rates.get(chosen, 0)
        reward = 1 if random.random() < prob else 0
        
        # 3. Update-uj bandit na osnovu dobijenog rezultata
        recommender.update(chosen, reward)
    
    # Nakon 1000 iteracija, možemo pogledati koliko često je prikazan svaki model i kakav je reward
    for item in recommender.items:
        print(f"Patika: {item}")
        print(f"  Ukupno prikazano: {recommender.counts[item]}")
        print(f"  Ukupno kupljeno: {recommender.rewards[item]}")
        if recommender.counts[item] > 0:
            print(f"  Prosečni reward: {recommender.rewards[item] / float(recommender.counts[item]):.3f}")
        print("---")


Patika: Nike Air Max
  Ukupno prikazano: 335
  Ukupno kupljeno: 14
  Prosečni reward: 0.042
---
Patika: Adidas Ultraboost
  Ukupno prikazano: 610
  Ukupno kupljeno: 42
  Prosečni reward: 0.069
---
Patika: Puma Rider
  Ukupno prikazano: 55
  Ukupno kupljeno: 2
  Prosečni reward: 0.036
---
